In [1]:
from poem.instance_creation_factories.triples_factory import TriplesFactory
from poem.models.unimodal.complex_cwa import ComplexCWA
from torch import optim
import numpy as np
from poem.training.cwa import CWATrainingLoop
import os
import logging
import sys
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

**Step 1: Create instances**

In [2]:
path_to_training_data = '../tests/resources/test.txt'

In [3]:
import timeit

In [4]:
factory = TriplesFactory(path=path_to_training_data)

**Step 2: Configure KGE model**

In [5]:
kge_model = ComplexCWA(triples_factory=factory,
                       embedding_dim=200,
                       input_dropout=0.2
                      )

**Step 3: Configure Training Loop**

In [6]:
optimizer = optim.Adam(params=kge_model.get_grad_params())
cwa_training_loop = CWATrainingLoop(model=kge_model, optimizer=optimizer)

**Step 4: Train**

In [7]:
losses = cwa_training_loop.train(num_epochs=5,
                                 batch_size=256)

Training epoch on cuda: 100%|██████████| 5/5 [00:19<00:00,  3.90s/it]


In [8]:
losses

[0.6931470672181946,
 0.6795057056321299,
 0.287819335218481,
 0.06256443300240072,
 0.0268778544551835]

In [9]:
from poem.evaluation import RankBasedEvaluator

In [10]:
evaluator = RankBasedEvaluator(kge_model, filter_neg_triples=True)

In [11]:
test_triples = factory.map_triples_to_id(path_to_triples=path_to_training_data)

In [12]:
evaluator.evaluate(test_triples)

Evaluating triples: 100%|██████████| 59071/59071 [03:26<00:00, 285.85it/s]


MetricResults(mean_rank=5262.496529599973, mean_reciprocal_rank=6.771512247972779e-05, adjusted_mean_rank=5262.496538064363, adjusted_mean_reciprocal_rank=0.0006810930470736689, hits_at_k={1: 6.771512247972779e-05, 3: 0.0001269658546494896, 5: 0.00026239609960894514, 10: 0.0005332565895278563})